In [1]:
from raw_schema import ProgramMovementSoloist, ProgramMovement, Program, Concert, Movement, Work
from sqlalchemy import create_engine

from sqlalchemy.orm import sessionmaker

from lookup_tables import Orchestra, EventType, Composer, Venue, Conductor, Soloist

import pandas as pd
import numpy as np

In [2]:
engine = create_engine("sqlite:///db.db")

In [166]:
query = """
SELECT pm.id, pm.program_id, con.id as concert_id, pm.movement_id, w.composer_id, c.name as composer_name, m.work_id, w.title as work_title
FROM program_movement pm 
    inner join movement m on m.id=pm.movement_id
    inner join work w on m.work_id=w.id
    inner join composer c on w.composer_id=c.id
    inner join concert con on pm.program_id=con.program_id
    inner join eventtype e on con.eventtype_id=e.id
    inner join program p on pm.program_id=p.id
where 
    e.name = 'Subscription Season'
"""

In [167]:
df = pd.read_sql_query(query, engine)

In [173]:
grouped = df[['program_id','concert_id','composer_id']]\
    .drop_duplicates()\
    .groupby(['program_id', 'concert_id'])

In [327]:
full_data = []
i = 0
for pid, data in grouped:
    d = list(data.composer_id.values)
    d += [0]
    full_data += d

In [328]:
lookup_d1 = {}
last = 0
for this in full_data[-1:0:-1]:
    if last in lookup_d1.keys():
        lookup_d1[last].append(this)
    else:
        lookup_d1[last] = [this]
    last = this
    

In [329]:
lookup_d2 = {}
lastval = 0
thisval = full_data[1]
for nextval in full_data[-1:0:-1]:
    if (lastval, thisval) in lookup_d1.keys():
        lookup_d2[(lastval, thisval)].append(nextval)
    else:
        lookup_d2[(lastval, thisval)] = [nextval]
    lastval = thisval
    thisval = nextval
    

In [330]:
compnames = engine.execute("select id, name from composer;").fetchall()
compnames = {id: name for id, name in compnames}
compnames[4] = 'INTERMISSION'
compnames[0] = 'BEGIN'

In [331]:
def make_composer_program():
    lastval = 0
    nextval = np.random.choice(lookup_d1[lastval])
    
    had_intermission = False
    
    while nextval != 0:
        currval = nextval
        print(compnames[currval])
        
        nextval = np.random.choice(lookup_d2[(lastval, currval)])
        
        if had_intermission and nextval == 4:
            while nextval == 4:
                nextval = np.random.choice(lookup_d2[(lastval, currval)])
        if nextval == 4:
            had_intermission = True
        
        lastval = currval

In [345]:
make_composer_program()

Sibelius, Jean
INTERMISSION
Mendelssohn, Felix
